In [1]:
import sys
import io
import time
import requests
import datetime
import numpy as np
import pandas as pd

path_root = '../..'
sys.path.append(f'{path_root}/')
from src.util.functions import Functions as F

# Disable warning
pd.options.mode.chained_assignment = None 

level = 'regioni'

path_data = f'{path_root}/data'
path_csv = f'{path_data}/csv'

csv_out_name = f'{path_csv}/covid19-{level}.csv'
pop_regions_name = f'{path_csv}/popolazione-regioni.csv'
icu_regions_name = f'{path_csv}/terapie-intensive.csv'
regions_name = f'{path_csv}/regioni.csv'

In [2]:
url_cum = F.get_url_cum(level)
url_cum

'https://raw.githubusercontent.com/pcm-dpc/COVID-19/master/dati-regioni/dpc-covid19-ita-regioni.csv'

In [3]:
df_raw_r = pd.read_csv(url_cum)
df_raw_r

,data,stato,codice_regione,denominazione_regione,lat,long,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,...,note,ingressi_terapia_intensiva,note_test,note_casi,totale_positivi_test_molecolare,totale_positivi_test_antigenico_rapido,tamponi_test_molecolare,tamponi_test_antigenico_rapido,codice_nuts_1,codice_nuts_2
0,2020-02-24T18:00:00,ITA,13,Abruzzo,42.351222,13.398438,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2020-02-24T18:00:00,ITA,17,Basilicata,40.639471,15.805148,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2020-02-24T18:00:00,ITA,18,Calabria,38.905976,16.594402,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2020-02-24T18:00:00,ITA,15,Campania,40.839566,14.250850,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2020-02-24T18:00:00,ITA,8,Emilia-Romagna,44.494367,11.341721,10,2,12,6,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13729,2021-12-08T17:00:00,ITA,19,Sicilia,38.115697,13.362357,320,46,366,13643,...,DECEDUTI: N. 5 IL 07/12/21 - N. 3 IL 06/12/21 ...,2.0,NaN,NaN,329979.0,0.0,3595101.0,4043550.0,ITG,ITG1
13730,2021-12-08T17:00:00,ITA,9,Toscana,43.769231,11.255889,270,47,317,10690,...,NaN,1.0,NaN,NaN,300881.0,5677.0,5117180.0,3100622.0,ITI,ITI1
13731,2021-12-08T17:00:00,ITA,10,Umbria,43.106758,12.388247,42,7,49,1930,...,NaN,1.0,NaN,NaN,68553.0,0.0,1265261.0,1290033.0,ITI,ITI2
13732,2021-12-08T17:00:00,ITA,2,Valle d'Aosta,45.737503,7.320149,22,1,23,708,...,NaN,0.0,NaN,NaN,12364.0,1248.0,119339.0,186176.0,ITC,ITC2


In [4]:
list(df_raw_r.columns)

['data',
 'stato',
 'codice_regione',
 'denominazione_regione',
 'lat',
 'long',
 'ricoverati_con_sintomi',
 'terapia_intensiva',
 'totale_ospedalizzati',
 'isolamento_domiciliare',
 'totale_positivi',
 'variazione_totale_positivi',
 'nuovi_positivi',
 'dimessi_guariti',
 'deceduti',
 'casi_da_sospetto_diagnostico',
 'casi_da_screening',
 'totale_casi',
 'tamponi',
 'casi_testati',
 'note',
 'ingressi_terapia_intensiva',
 'note_test',
 'note_casi',
 'totale_positivi_test_molecolare',
 'totale_positivi_test_antigenico_rapido',
 'tamponi_test_molecolare',
 'tamponi_test_antigenico_rapido',
 'codice_nuts_1',
 'codice_nuts_2']

In [5]:
# Drop useless columns
cols_to_drop = ['stato', 'lat', 'long', 'casi_da_sospetto_diagnostico', 'casi_da_screening', 'note', 'note_test', 'note_casi']
df_rd = df_raw_r.drop(cols_to_drop, axis=1)
df_rd

,data,codice_regione,denominazione_regione,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,totale_positivi,variazione_totale_positivi,nuovi_positivi,...,totale_casi,tamponi,casi_testati,ingressi_terapia_intensiva,totale_positivi_test_molecolare,totale_positivi_test_antigenico_rapido,tamponi_test_molecolare,tamponi_test_antigenico_rapido,codice_nuts_1,codice_nuts_2
0,2020-02-24T18:00:00,13,Abruzzo,0,0,0,0,0,0,0,...,0,5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2020-02-24T18:00:00,17,Basilicata,0,0,0,0,0,0,0,...,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2020-02-24T18:00:00,18,Calabria,0,0,0,0,0,0,0,...,0,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2020-02-24T18:00:00,15,Campania,0,0,0,0,0,0,0,...,0,10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2020-02-24T18:00:00,8,Emilia-Romagna,10,2,12,6,18,0,18,...,18,148,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13729,2021-12-08T17:00:00,19,Sicilia,320,46,366,13643,14009,-101,618,...,329979,7638651,2791362.0,2.0,329979.0,0.0,3595101.0,4043550.0,ITG,ITG1
13730,2021-12-08T17:00:00,9,Toscana,270,47,317,10690,11007,330,791,...,306558,8217802,3392315.0,1.0,300881.0,5677.0,5117180.0,3100622.0,ITI,ITI1
13731,2021-12-08T17:00:00,10,Umbria,42,7,49,1930,1979,18,179,...,68553,2555294,477815.0,1.0,68553.0,0.0,1265261.0,1290033.0,ITI,ITI2
13732,2021-12-08T17:00:00,2,Valle d'Aosta,22,1,23,708,731,-4,60,...,13612,305515,99967.0,0.0,12364.0,1248.0,119339.0,186176.0,ITC,ITC2


In [6]:
# Compute most recent couple of dates
last_date_str = df_rd.iloc[-1].data.split('T')[0]
last_date = datetime.datetime.strptime(last_date_str, '%Y-%m-%d')
prev_date = F.get_previous_date(last_date)
prev_date_str = datetime.datetime.strftime(prev_date, '%Y-%m-%d')
last_date_str, prev_date_str

('2021-12-08', '2021-12-07')

In [7]:
# Current state S
df_raw_r_1 = df_rd[df_rd.data.str.startswith(last_date_str)].sort_values(by='codice_regione').reset_index(drop=True)
df_raw_r_1

,data,codice_regione,denominazione_regione,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,totale_positivi,variazione_totale_positivi,nuovi_positivi,...,totale_casi,tamponi,casi_testati,ingressi_terapia_intensiva,totale_positivi_test_molecolare,totale_positivi_test_antigenico_rapido,tamponi_test_molecolare,tamponi_test_antigenico_rapido,codice_nuts_1,codice_nuts_2
0,2021-12-08T17:00:00,1,Piemonte,453,43,496,14351,14847,556,1165,...,407850,9889170,2804282.0,8.0,373930.0,33920.0,3910143.0,5979027.0,ITC,ITC1
1,2021-12-08T17:00:00,2,Valle d'Aosta,22,1,23,708,731,-4,60,...,13612,305515,99967.0,0.0,12364.0,1248.0,119339.0,186176.0,ITC,ITC2
2,2021-12-08T17:00:00,3,Lombardia,1066,136,1202,41151,42353,2227,3373,...,953255,21509933,6267164.0,10.0,865045.0,88210.0,12168237.0,9341696.0,ITC,ITC4
3,2021-12-08T17:00:00,5,Veneto,667,120,787,41778,42565,1917,3516,...,538767,17229454,2340523.0,3.0,518502.0,20265.0,7772501.0,9456953.0,ITH,ITH3
4,2021-12-08T17:00:00,6,Friuli Venezia Giulia,300,27,327,7728,8055,79,763,...,136401,4033280,919866.0,2.0,118807.0,17594.0,2480744.0,1552536.0,ITH,ITH4
5,2021-12-08T17:00:00,7,Liguria,195,27,222,6007,6229,501,690,...,126523,3148827,980875.0,2.0,126523.0,0.0,1857320.0,1291507.0,ITC,ITC3
6,2021-12-08T17:00:00,8,Emilia-Romagna,837,82,919,26796,27715,817,1391,...,467571,10782457,2282152.0,7.0,467115.0,456.0,6771414.0,4011043.0,ITH,ITH5
7,2021-12-08T17:00:00,9,Toscana,270,47,317,10690,11007,330,791,...,306558,8217802,3392315.0,1.0,300881.0,5677.0,5117180.0,3100622.0,ITI,ITI1
8,2021-12-08T17:00:00,10,Umbria,42,7,49,1930,1979,18,179,...,68553,2555294,477815.0,1.0,68553.0,0.0,1265261.0,1290033.0,ITI,ITI2
9,2021-12-08T17:00:00,11,Marche,112,30,142,5521,5663,191,464,...,127191,1808360,1045408.0,3.0,127191.0,0.0,1526687.0,281673.0,ITI,ITI3


In [8]:
# State S - 1
df_raw_r_0 = df_rd[df_rd.data.str.startswith(prev_date_str)].sort_values(by='codice_regione').reset_index(drop=True)
df_raw_r_0

,data,codice_regione,denominazione_regione,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,totale_positivi,variazione_totale_positivi,nuovi_positivi,...,totale_casi,tamponi,casi_testati,ingressi_terapia_intensiva,totale_positivi_test_molecolare,totale_positivi_test_antigenico_rapido,tamponi_test_molecolare,tamponi_test_antigenico_rapido,codice_nuts_1,codice_nuts_2
0,2021-12-07T17:00:00,1,Piemonte,463,39,502,13789,14291,594,1310,...,406685,9846743,2797983.0,2.0,373227.0,33458.0,3900070.0,5946673.0,ITC,ITC1
1,2021-12-07T17:00:00,2,Valle d'Aosta,20,1,21,714,735,-28,57,...,13552,303102,99628.0,0.0,12320.0,1232.0,119068.0,184034.0,ITC,ITC2
2,2021-12-07T17:00:00,3,Lombardia,1041,134,1175,38951,40126,505,2783,...,949882,21397441,6252006.0,16.0,862962.0,86920.0,12136839.0,9260602.0,ITC,ITC4
3,2021-12-07T17:00:00,5,Veneto,705,124,829,39819,40648,1410,2960,...,535251,17135603,2334346.0,15.0,515222.0,20029.0,7746697.0,9388906.0,ITH,ITH3
4,2021-12-07T17:00:00,6,Friuli Venezia Giulia,303,28,331,7645,7976,-59,505,...,135649,4009148,917904.0,3.0,118171.0,17478.0,2471942.0,1537206.0,ITH,ITH4
5,2021-12-07T17:00:00,7,Liguria,192,26,218,5510,5728,190,483,...,125833,3130763,976567.0,2.0,125833.0,0.0,1850874.0,1279889.0,ITC,ITC3
6,2021-12-07T17:00:00,8,Emilia-Romagna,807,81,888,26010,26898,799,1350,...,466182,10746250,2277532.0,11.0,465726.0,456.0,6749159.0,3997091.0,ITH,ITH5
7,2021-12-07T17:00:00,9,Toscana,266,46,312,10365,10677,126,498,...,305767,8185244,3382905.0,2.0,300135.0,5632.0,5105540.0,3079704.0,ITI,ITI1
8,2021-12-07T17:00:00,10,Umbria,40,8,48,1913,1961,77,141,...,68374,2544504,476940.0,1.0,68374.0,0.0,1262205.0,1282299.0,ITI,ITI2
9,2021-12-07T17:00:00,11,Marche,105,31,136,5336,5472,161,411,...,126727,1801256,1041443.0,2.0,126727.0,0.0,1521648.0,279608.0,ITI,ITI3


In [9]:
# New df regions state S with Trentino-Alto Adige
df_r_1 = df_raw_r_1
# df_r_1 = Functions.replace_bt_with_taa(df_raw_r_1)
# df_r_1.sort_values(by='codice_regione').reset_index(drop=True)

In [10]:
# New df regions state S - 1 with Trentino-Alto Adige
df_r_0 = df_raw_r_0
# df_r_0 = Functions.replace_bt_with_taa(df_raw_r_0)
# df_r_0.sort_values(by='codice_regione').reset_index()

In [11]:
df_r = df_r_1.copy()
# Merge with population data
df_pop_r = pd.read_csv(pop_regions_name)
df_m_r_1 = df_r.merge(df_pop_r, on='denominazione_regione')
df_m_r_1.sort_values(by='codice_regione').reset_index(drop=True)

,data,codice_regione,denominazione_regione,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,totale_positivi,variazione_totale_positivi,nuovi_positivi,...,tamponi,casi_testati,ingressi_terapia_intensiva,totale_positivi_test_molecolare,totale_positivi_test_antigenico_rapido,tamponi_test_molecolare,tamponi_test_antigenico_rapido,codice_nuts_1,codice_nuts_2,popolazione
0,2021-12-08T17:00:00,1,Piemonte,453,43,496,14351,14847,556,1165,...,9889170,2804282.0,8.0,373930.0,33920.0,3910143.0,5979027.0,ITC,ITC1,4.341375e+06
1,2021-12-08T17:00:00,2,Valle d'Aosta,22,1,23,708,731,-4,60,...,305515,99967.0,0.0,12364.0,1248.0,119339.0,186176.0,ITC,ITC2,1.255010e+05
2,2021-12-08T17:00:00,3,Lombardia,1066,136,1202,41151,42353,2227,3373,...,21509933,6267164.0,10.0,865045.0,88210.0,12168237.0,9341696.0,ITC,ITC4,1.010397e+07
3,2021-12-08T17:00:00,5,Veneto,667,120,787,41778,42565,1917,3516,...,17229454,2340523.0,3.0,518502.0,20265.0,7772501.0,9456953.0,ITH,ITH3,4.907704e+06
4,2021-12-08T17:00:00,6,Friuli Venezia Giulia,300,27,327,7728,8055,79,763,...,4033280,919866.0,2.0,118807.0,17594.0,2480744.0,1552536.0,ITH,ITH4,1.211357e+06
5,2021-12-08T17:00:00,7,Liguria,195,27,222,6007,6229,501,690,...,3148827,980875.0,2.0,126523.0,0.0,1857320.0,1291507.0,ITC,ITC3,1.543127e+06
6,2021-12-08T17:00:00,8,Emilia-Romagna,837,82,919,26796,27715,817,1391,...,10782457,2282152.0,7.0,467115.0,456.0,6771414.0,4011043.0,ITH,ITH5,4.467118e+06
7,2021-12-08T17:00:00,9,Toscana,270,47,317,10690,11007,330,791,...,8217802,3392315.0,1.0,300881.0,5677.0,5117180.0,3100622.0,ITI,ITI1,3.722729e+06
8,2021-12-08T17:00:00,10,Umbria,42,7,49,1930,1979,18,179,...,2555294,477815.0,1.0,68553.0,0.0,1265261.0,1290033.0,ITI,ITI2,8.802850e+05
9,2021-12-08T17:00:00,11,Marche,112,30,142,5521,5663,191,464,...,1808360,1045408.0,3.0,127191.0,0.0,1526687.0,281673.0,ITI,ITI3,1.518400e+06


In [12]:
# Update icu csv data
df_icu_updated_r = pd.DataFrame.from_dict(F.icu_data())
df_icu_updated_r.to_csv(icu_regions_name, index=False)

In [13]:
# Merge with icu data
df_icu_r = pd.read_csv(icu_regions_name)
df_m_r_2 = df_m_r_1.merge(df_icu_r, left_on='denominazione_regione', right_on='regione')
df_m_r_2.sort_values(by='codice_regione').reset_index(drop=True)

,data,codice_regione,denominazione_regione,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,totale_positivi,variazione_totale_positivi,nuovi_positivi,...,tamponi_test_antigenico_rapido,codice_nuts_1,codice_nuts_2,popolazione,regione,ricoverati_area_non_critica,posti_letto_area_non_critica,ricoverati_terapia_intensiva,posti_letto_terapia_intensiva,posti_letto_terapia_intensiva_attivabili
0,2021-12-08T17:00:00,1,Piemonte,453,43,496,14351,14847,556,1165,...,5979027.0,ITC,ITC1,4.341375e+06,Piemonte,453,5824,43,628,99
1,2021-12-08T17:00:00,2,Valle d'Aosta,22,1,23,708,731,-4,60,...,186176.0,ITC,ITC2,1.255010e+05,Valle d'Aosta,22,99,1,33,0
2,2021-12-08T17:00:00,3,Lombardia,1066,136,1202,41151,42353,2227,3373,...,9341696.0,ITC,ITC4,1.010397e+07,Lombardia,1066,7945,136,1530,0
3,2021-12-08T17:00:00,5,Veneto,667,120,787,41778,42565,1917,3516,...,9456953.0,ITH,ITH3,4.907704e+06,Veneto,667,6000,120,1000,0
4,2021-12-08T17:00:00,6,Friuli Venezia Giulia,300,27,327,7728,8055,79,763,...,1552536.0,ITH,ITH4,1.211357e+06,Friuli Venezia Giulia,300,1277,27,175,0
5,2021-12-08T17:00:00,7,Liguria,195,27,222,6007,6229,501,690,...,1291507.0,ITC,ITC3,1.543127e+06,Liguria,195,1688,27,221,12
6,2021-12-08T17:00:00,8,Emilia-Romagna,837,82,919,26796,27715,817,1391,...,4011043.0,ITH,ITH5,4.467118e+06,Emilia-Romagna,837,8336,82,889,0
7,2021-12-08T17:00:00,9,Toscana,270,47,317,10690,11007,330,791,...,3100622.0,ITI,ITI1,3.722729e+06,Toscana,270,5032,47,570,32
8,2021-12-08T17:00:00,10,Umbria,42,7,49,1930,1979,18,179,...,1290033.0,ITI,ITI2,8.802850e+05,Umbria,42,662,7,88,39
9,2021-12-08T17:00:00,11,Marche,112,30,142,5521,5663,191,464,...,281673.0,ITI,ITI3,1.518400e+06,Marche,112,966,30,250,55


In [14]:
# Copy last merged df into df_r
df_r = df_m_r_2.copy()

In [15]:
# Add columns
df_r['variazione_ricoverati_con_sintomi'] = df_r_1['ricoverati_con_sintomi'] - df_r_0['ricoverati_con_sintomi']
df_r['variazione_terapia_intensiva'] = df_r_1['terapia_intensiva'] - df_r_0['terapia_intensiva']
df_r['variazione_totale_ospedalizzati'] = df_r_1['totale_ospedalizzati'] - df_r_0['totale_ospedalizzati']
df_r['posti_disponibili_terapia_intensiva'] = df_r['posti_letto_terapia_intensiva'] - df_r['terapia_intensiva']
df_r['saturazione_terapia_intensiva'] = ((df_r_1['terapia_intensiva'] / df_r['posti_letto_terapia_intensiva']) * 100).round(2)
df_r['variazione_isolamento_domiciliare'] = df_r_1['isolamento_domiciliare'] - df_r_0['isolamento_domiciliare']
df_r['variazione_nuovi_positivi'] = df_r_1['nuovi_positivi'] - df_r_0['nuovi_positivi']
df_r['nuovi_tamponi'] = df_r_1['tamponi'] - df_r_0['tamponi']
df_r['nuovi_casi_testati'] = df_r_1['casi_testati'] - df_r_0['casi_testati']
df_r['nuovi_deceduti'] = df_r_1['deceduti'] - df_r_0['deceduti']
df_r['nuovi_dimessi_guariti'] = df_r_1['dimessi_guariti'] - df_r_0['dimessi_guariti']
df_r['np_su_nt'] = ((df_r['nuovi_positivi'] / df_r['nuovi_tamponi']) * 100).round(2)
df_r['np_su_p'] = ((df_r['nuovi_positivi'] / df_r['popolazione']) * 100).round(5)
df_r['tc_su_p'] = ((df_r['totale_casi'] / df_r['popolazione']) * 100).round(2)

df_r = df_r.sort_values(by='nuovi_positivi', ascending=False).reset_index(drop=True)
df_r

,data,codice_regione,denominazione_regione,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,totale_positivi,variazione_totale_positivi,nuovi_positivi,...,saturazione_terapia_intensiva,variazione_isolamento_domiciliare,variazione_nuovi_positivi,nuovi_tamponi,nuovi_casi_testati,nuovi_deceduti,nuovi_dimessi_guariti,np_su_nt,np_su_p,tc_su_p
0,2021-12-08T17:00:00,5,Veneto,667,120,787,41778,42565,1917,3516,...,12.00,1959,556,93851,6177.0,6,1593,3.75,0.07164,10.98
1,2021-12-08T17:00:00,3,Lombardia,1066,136,1202,41151,42353,2227,3373,...,8.89,2200,590,112492,15158.0,25,1121,3.00,0.03338,9.43
2,2021-12-08T17:00:00,12,Lazio,760,113,873,24566,25439,660,1554,...,11.98,662,80,53004,5588.0,3,891,2.93,0.02649,7.43
3,2021-12-08T17:00:00,8,Emilia-Romagna,837,82,919,26796,27715,817,1391,...,9.22,786,41,36207,4620.0,13,559,3.84,0.03114,10.47
4,2021-12-08T17:00:00,15,Campania,350,25,375,19259,19634,126,1175,...,4.81,115,25,32862,3915.0,1,1048,3.58,0.02031,8.64
5,2021-12-08T17:00:00,1,Piemonte,453,43,496,14351,14847,556,1165,...,6.85,562,-145,42427,6299.0,0,609,2.75,0.02683,9.39
6,2021-12-08T17:00:00,9,Toscana,270,47,317,10690,11007,330,791,...,8.25,325,293,32558,9410.0,5,456,2.43,0.02125,8.23
7,2021-12-08T17:00:00,6,Friuli Venezia Giulia,300,27,327,7728,8055,79,763,...,15.43,83,258,24132,1962.0,8,665,3.16,0.06299,11.26
8,2021-12-08T17:00:00,7,Liguria,195,27,222,6007,6229,501,690,...,12.22,497,207,18064,4308.0,5,184,3.82,0.04471,8.20
9,2021-12-08T17:00:00,19,Sicilia,320,46,366,13643,14009,-101,618,...,5.32,-94,-357,23329,5447.0,10,709,2.65,0.01244,6.64


In [16]:
# Write pre-processed data
df_r.to_csv(csv_out_name, index=False)

In [17]:
# ICU Stats
df_r[['denominazione_regione', 'terapia_intensiva', 'posti_disponibili_terapia_intensiva', 'posti_letto_terapia_intensiva', 'saturazione_terapia_intensiva']] \
    .sort_values(by='saturazione_terapia_intensiva', ascending=False).reset_index(drop=True)

,denominazione_regione,terapia_intensiva,posti_disponibili_terapia_intensiva,posti_letto_terapia_intensiva,saturazione_terapia_intensiva
0,P.A. Bolzano,17,83,100,17.00
1,P.A. Trento,15,75,90,16.67
2,Friuli Venezia Giulia,27,148,175,15.43
3,Liguria,27,194,221,12.22
4,Marche,30,220,250,12.00
5,Veneto,120,880,1000,12.00
6,Lazio,113,830,943,11.98
7,Calabria,17,164,181,9.39
8,Emilia-Romagna,82,807,889,9.22
9,Lombardia,136,1394,1530,8.89


In [18]:
def add_cols(df):
    df['variazione_terapia_intensiva'] = df['terapia_intensiva'].diff()
    df['variazione_ricoverati_con_sintomi'] = df['ricoverati_con_sintomi'].diff()
    df['variazione_totale_ospedalizzati'] = df['totale_ospedalizzati'].diff()
    df['nuovi_dimessi_guariti'] = df['dimessi_guariti'].diff()
    df['nuovi_deceduti'] = df['deceduti'].diff()
    df['nuovi_tamponi'] = df['tamponi'].diff()
    df['nuovi_casi_testati'] = df['casi_testati'].diff()
    df['variazione_isolamento_domiciliare'] = df['isolamento_domiciliare'].diff()
    df = df.iloc[1:]
    df['ti_su_to'] = ((df['terapia_intensiva'] / df['totale_ospedalizzati']).replace(np.inf, 0) * 100).round(2)
    df['to_su_tp'] = ((df['totale_ospedalizzati'] / df['totale_positivi']).replace(np.inf, 0) * 100).round(2)
    df['np_su_nt'] = ((df['nuovi_positivi'] / df['nuovi_tamponi']).replace(np.inf, 0) * 100).round(2)
    df['np_su_nct'] = ((df['nuovi_positivi'] / df['nuovi_casi_testati']).replace(np.inf, 0) * 100).round(2)
    df['var_ti_perc'] = ((df['variazione_terapia_intensiva'] / (df['terapia_intensiva'] - df['variazione_terapia_intensiva'])).replace(np.inf, 0) * 100).round(2)
    df['var_pos_perc'] = ((df['variazione_totale_positivi'] / (df['totale_positivi'] - df['variazione_totale_positivi'])).replace(np.inf, 0) * 100).round(2)
    return df.fillna(0)

In [19]:
# df_rd.sort_values(by='codice_regione').reset_index(drop=True)
df_rn = pd.read_csv(regions_name)
for region_code in df_rn.codice_regione:
    if region_code != 4: # TAA
        df_cur_region = df_rd[df_rd.codice_regione == region_code]
        region_name = df_cur_region.denominazione_regione.iloc[0].lower()
        csv_out_region_name = '{path_csv}/regioni/covid19-{region_name}.csv'.format(path_csv=path_csv, region_name=region_name)
        df_cur_region = add_cols(df_cur_region)
        df_cur_region.to_csv(csv_out_region_name, index=False)
        print('Aggiornato {file}'.format(file=csv_out_region_name))

Aggiornato ../../data/csv/regioni/covid19-piemonte.csv
Aggiornato ../../data/csv/regioni/covid19-valle d'aosta.csv
Aggiornato ../../data/csv/regioni/covid19-lombardia.csv
Aggiornato ../../data/csv/regioni/covid19-veneto.csv
Aggiornato ../../data/csv/regioni/covid19-friuli venezia giulia.csv
Aggiornato ../../data/csv/regioni/covid19-liguria.csv
Aggiornato ../../data/csv/regioni/covid19-emilia-romagna.csv
Aggiornato ../../data/csv/regioni/covid19-toscana.csv
Aggiornato ../../data/csv/regioni/covid19-umbria.csv
Aggiornato ../../data/csv/regioni/covid19-marche.csv
Aggiornato ../../data/csv/regioni/covid19-lazio.csv
Aggiornato ../../data/csv/regioni/covid19-abruzzo.csv
Aggiornato ../../data/csv/regioni/covid19-molise.csv
Aggiornato ../../data/csv/regioni/covid19-campania.csv
Aggiornato ../../data/csv/regioni/covid19-puglia.csv
Aggiornato ../../data/csv/regioni/covid19-basilicata.csv
Aggiornato ../../data/csv/regioni/covid19-calabria.csv
Aggiornato ../../data/csv/regioni/covid19-sicilia.csv


In [20]:
# df_tb = df_rd[df_rd.codice_regione > 20].reset_index(drop=True)
# df_tb

In [21]:
# Initialize df_taa
# df_taa = Functions.replace_bt_with_taa(df_tb[:2])
# for i in range(2, df_tb.shape[0], 2):
#     df_taa = df_taa.append(Functions.replace_bt_with_taa(df_tb[i:i+2]), ignore_index=True)
# df_taa = add_cols(df_taa)
# df_taa

In [22]:
# region_name = df_taa.denominazione_regione.iloc[0].lower()
# csv_out_region_name = '{path_csv}/regioni/covid19-{region_name}.csv'.format(path_csv=path_csv, region_name=region_name)
# df_taa.to_csv(csv_out_region_name, index=False)
# print('Aggiornato {file}'.format(file=csv_out_region_name))